In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os

In [ ]:
severity_code = {'MILD': 0,
                'MODERATE': 1,
                'NORMAL': 2,
                'SEVERE': 3}

In [ ]:
# specifying file locations

source_folder_train = '/Users/aravdhoot/Remote-PD-Detection/split_energy_images/Train/'
source_folder_val = '/Users/aravdhoot/Remote-PD-Detection/split_energy_images/Validation/'
channels = 3

In [ ]:
# adding the image arrays to an array

resized_images_train = [
    cv2.resize(cv2.imread(os.path.join(source_folder_train, severity, video, image)), (224, 224))
    for severity in os.listdir(source_folder_train)
    for video in os.listdir(os.path.join(source_folder_train, severity))
    for image in os.listdir(os.path.join(source_folder_train, severity, video))
]

# formatting the array

num_samples = len(resized_images_train)
input_shape = (num_samples, 224, 224, channels)
input_array_train = np.empty(input_shape, dtype=np.uint8)

In [ ]:
# adding the corresponding severity codes to the array

input_array_train = np.array(resized_images_train)
print(input_array_train.shape)
x_train = input_array_train

y_train = [
    severity_code[severity]
    for severity in os.listdir(source_folder_train)
    for video in os.listdir(os.path.join(source_folder_train, severity))
    for image in os.listdir(os.path.join(source_folder_train, severity, video))
]

In [ ]:
# adding the image arrays to an array

resized_images_val = [
    cv2.resize(cv2.imread(os.path.join(source_folder_val, severity, video, image)), (224, 224))
    for severity in os.listdir(source_folder_val)
    for video in os.listdir(os.path.join(source_folder_val, severity))
    for image in os.listdir(os.path.join(source_folder_val, severity, video))
]

# formatting the array

num_samples = len(resized_images_val)
input_shape = (num_samples, 224, 224, channels)
input_array = np.empty(input_shape, dtype=np.uint8)

In [ ]:
# adding the corresponding severity codes to the array

input_array_val = np.array(resized_images_val)
print(input_array.shape)
x_val = input_array

y_val = [
    severity_code[severity]
    for severity in os.listdir(source_folder_val)
    for video in os.listdir(os.path.join(source_folder_val, severity))
    for image in os.listdir(os.path.join(source_folder_val, severity, video))
]

In [ ]:
# randomly oversampling all the training images to have the same number of images

ros = RandomOverSampler(random_state=42)
x_train_2d = x_train.reshape(x_train.shape[0], -1)
X_resampled, y_resampled = ros.fit_resample(x_train_2d, y_train)
x_resampled_original = X_resampled.reshape(-1, x_train.shape[1], x_train.shape[2], x_train.shape[3])

In [ ]:
# applying one-hot encoding on the class values

encoder = OneHotEncoder(sparse=False)
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1))
y_train = encoder.fit_transform(np.array(y_resampled).reshape(-1,1))

In [ ]:
# normalizing the image arrays

x_resampled_original = x_resampled_original/255.0
x_val = x_val/255.0

In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
base_model = ResNet50(weights=None, include_top=False, input_shape=(224, 224, 3))

# Define the custom top layers for your classification task
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(4, activation='softmax')  # Adjust the number of units to match the number of classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model summary to check architecture
model.summary()

In [ ]:
x_train = x_resampled_original

In [ ]:
epochs1 = 36
epochs2= 4
batch_size = 32

x_val = x_val/255.0
chs=2000
history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}
nchunk=x_train.shape[0]//chs
for i in range(0,nchunk+1):
    rl = i*chs
    rr = min((i+1)*chs, x_train.shape[0])
    print('doing chunk', rl,rr)
    x_traini = x_train[rl:rr,:,:]/255.0
    y_traini = y_train[rl:rr]
    historyi = model.fit(x_traini, y_traini, epochs=epochs1, batch_size=batch_size)
    historyi = model.fit(x_traini, y_traini, epochs=epochs2, batch_size=batch_size, validation_data=(x_val, y_val))
    history['accuracy'] = history['accuracy'] +historyi.history['accuracy'] 
    history['val_accuracy'] = history['val_accuracy'] +historyi.history['val_accuracy'] 
    history['loss'] = history['loss'] +historyi.history['loss']
    history['val_loss'] = history['val_loss'] +historyi.history['val_loss']